# Data cleaning and merging for the Viz Project

In [209]:
import pandas as pd

In [210]:
dogs = pd.read_csv('hunde-vie.csv', sep=';', skiprows=1)
population_density = pd.read_csv('vie-bez-biz-pop-den-2002f.csv', sep=';', skiprows=1)
population = pd.read_csv('vie-bez-pop-sex-age5-stk-ori-geo4-2002f.csv', sep=';', skiprows=1)
dog_breeds = pd.read_csv('dog_breeds.csv', sep=',', skiprows=0)
breeds_mapping = pd.read_csv('dog_mappings_two.csv', sep=',', skiprows=0)


In [211]:
dogs = dogs.drop(columns=["NUTS1", "NUTS2", "NUTS3", "SUB_DISTRICT_CODE", "Postal_CODE", "Ref_Date"])
dogs.head()

,DISTRICT_CODE,Dog Breed,Anzahl
0,90100,Afghanischer Windhund,1
1,90100,Amerikanischer Cockerspaniel,1
2,90100,Amerikanischer Staffordshire-Terrier,2
3,90100,Australian Shepherd Dog,2
4,90100,Australian Terrier,1


In [212]:
dogs["Breed_single"] = dogs["Dog Breed"].str.split("/").str[0].str.strip()
dogs["Breed_single"].unique()

array(['Afghanischer Windhund', 'Amerikanischer Cockerspaniel',
       'Amerikanischer Staffordshire-Terrier', 'Australian Shepherd Dog',
       'Australian Terrier', 'Bayerischer Gebirgsschweisshund', 'Beagle',
       'Bearded Collie', 'Bergamasker Hirtenhund', 'Berner Sennenhund',
       'Bernhardiner', 'Bichon Frisé', 'Bolonka Zwetna', 'Border-Collie',
       'Boston Terrier', 'Boxer', 'Bracke', 'Bulldog', 'Bullterrier',
       'Cairn-Terrier', 'Cao de Agua Portugues',
       'Cavalier King Charles Spaniel', 'Chihuahua kurzhaariger Schlag',
       'Chihuahua langhaariger Schlag',
       'Chinesischer Schopfhund Hairless-Schlag', 'Chow-Chow',
       'Cocker Spaniel', 'Cockerpoo', 'Collie', 'Dackel', 'Dalmatiner',
       'Deutsche Dogge', 'Deutscher Jagdterrier', 'Deutscher Schäferhund',
       'Dogo Argentino', 'Drahthaar-Foxterrier',
       'Englischer Cockerspaniel', 'English Springer Spaniel',
       'Entlebucher Sennenhund', 'Flat Coated Retriever',
       'Französische Bulldogge

In [213]:
dogs_grouped = dogs.groupby(["DISTRICT_CODE", "Breed_single"])["Anzahl"].sum().reset_index()

# breeds = dogs_grouped.groupby("Breed_single", as_index = False).sum()
# breeds["Breed_single"].to_csv('dog_breed_german_names.csv', index = False)

In [214]:
population_density = population_density[population_density["REF_YEAR"] == 2012]
population_density = population_density.loc[:, ~population_density.columns.str.contains("^Unnamed")]
population_density = population_density.drop(columns=["NUTS", "SUB_DISTRICT_CODE", "REF_YEAR", "REF_DATE"])
population_density.head()


,DISTRICT_CODE,POP_VALUE,AREA,POP_DENSITY
10,90000,1.717.084,"414,87","4138,84"
33,90100,16.290,"2,87","5678,38"
56,90200,96.582,"19,24","5019,33"
79,90300,84.574,"7,40","11432,41"
102,90400,30.724,"1,78","17307,27"


In [215]:
population.head()

,NUTS,DISTRICT_CODE,SUB_DISTRICT_CODE,REF_YEAR,REF_DATE,SEX,AGE5,AUT,EEA,REU,TCN
0,AT13,90100,90100,2002,20020101,1,1,250,35,19,16
1,AT13,90100,90100,2002,20020101,1,2,225,18,21,19
2,AT13,90100,90100,2002,20020101,1,3,212,24,11,17
3,AT13,90100,90100,2002,20020101,1,4,268,47,28,18
4,AT13,90100,90100,2002,20020101,1,5,302,68,35,30


In [216]:
# 1. Filter by year
df = population[population["REF_YEAR"] == 2012].copy()

# 2. Drop sex
df = df.drop(columns=["SEX"])

# 3. Group by region/date/age and sum over populations
grouped = df.groupby(
    ["NUTS", "DISTRICT_CODE", "SUB_DISTRICT_CODE", "REF_YEAR", "REF_DATE", "AGE5"]
).sum().reset_index()

# 4. Total population per row (all groups)
grouped["TOTAL"] = grouped[["AUT", "EEA", "REU", "TCN"]].sum(axis=1)

# 5. Weighted average age per region/date
def weighted_avg_age(group):
    return (group["AGE5"] * group["TOTAL"] * 5).sum() / group["TOTAL"].sum()

avg_age = grouped.groupby(
    ["NUTS", "DISTRICT_CODE", "SUB_DISTRICT_CODE", "REF_YEAR", "REF_DATE"]
).apply(weighted_avg_age).reset_index(name="AVG_AGE")

# 6. AUT share vs. others
def aut_share(group):
    aut = group["AUT"].sum()
    others = group[["EEA", "REU", "TCN"]].sum().sum()
    return aut / (aut + others)

aut_ratio = grouped.groupby(
    ["NUTS", "DISTRICT_CODE", "SUB_DISTRICT_CODE", "REF_YEAR", "REF_DATE"]
).apply(aut_share).reset_index(name="AUT_RATIO")

# 7. Merge results
population_reshape = avg_age.merge(aut_ratio, on=["NUTS", "DISTRICT_CODE", "SUB_DISTRICT_CODE", "REF_YEAR", "REF_DATE"])
population_reshape = population_reshape.drop(columns=["NUTS", "SUB_DISTRICT_CODE", "REF_YEAR", "REF_DATE"])
population_reshape


/tmp/ipykernel_16417/716251150.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ).apply(weighted_avg_age).reset_index(name="AVG_AGE")
/tmp/ipykernel_16417/716251150.py:31: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  ).apply(aut_share).reset_index(name="AUT_RATIO")


,DISTRICT_CODE,AVG_AGE,AUT_RATIO
0,90100,49.169122,0.666605
1,90200,42.342051,0.583929
2,90300,44.029785,0.646038
3,90400,44.420486,0.639988
4,90500,42.302638,0.576520
5,90600,43.765483,0.654587
6,90700,43.248065,0.660875
7,90800,43.418081,0.676868
8,90900,43.257589,0.649667
9,91000,42.921063,0.611372


In [217]:
dogs_pop = dogs_grouped.merge(population_density).merge(population_reshape)

In [ ]:
full_breeds = breeds_mapping.merge(dog_breeds, left_on=["English name"], right_on=["Breed Name"], how='left')
full_breeds2 = breeds_mapping.merge(dog_breeds, left_on=["English name"], right_on=["Breed Name"], how='outer')
# full_breeds2.to_csv('breed_info.csv', index = False)

In [219]:
dogs_pop_breed = dogs_pop.merge(full_breeds, left_on=["Breed_single"], right_on=["German Breed"], how='left')

In [220]:

dogs_pop_breed
dogs_pop_breed['POP_DENSITY'] = dogs_pop_breed['POP_DENSITY'].str.replace(',', '.').astype('float64')
dogs_pop_breed['POP_VALUE'] = dogs_pop_breed['POP_VALUE'].str.replace(',', '.').astype('float64')
dogs_pop_breed['AREA'] = dogs_pop_breed['AREA'].str.replace(',', '.').astype('float64')
dogs_pop_breed['AVG_AGE'] = dogs_pop_breed['AVG_AGE'].round(1)
dogs_pop_breed['AUT_RATIO'] = dogs_pop_breed['AUT_RATIO'].round(3)

In [221]:
dogs_pop_breed_der = dogs_pop_breed.assign(DOG_DENSITY=lambda x: x['Anzahl'] / x['POP_VALUE'])
dogs_pop_breed_der['DOG_DENSITY'] = dogs_pop_breed_der['DOG_DENSITY'].round(2)

In [222]:
print(dogs_pop_breed_der.columns)

Index(['DISTRICT_CODE', 'Breed_single', 'Anzahl', 'POP_VALUE', 'AREA',
       'POP_DENSITY', 'AVG_AGE', 'AUT_RATIO', 'German Breed', 'English name',
       'Breed Name', 'Detailed Description Link', 'Dog Size',
       'Dog Breed Group', 'Height', 'Avg. Height, cm', 'Weight',
       'Avg. Weight, kg', 'Life Span', 'Avg. Life Span, years', 'Adaptability',
       'Adapts Well To Apartment Living', 'Good For Novice Owners',
       'Sensitivity Level', 'Tolerates Being Alone', 'Tolerates Cold Weather',
       'Tolerates Hot Weather', 'All Around Friendliness',
       'Affectionate With Family', 'Kid-Friendly', 'Dog Friendly',
       'Friendly Toward Strangers', 'Health And Grooming Needs',
       'Amount Of Shedding', 'Drooling Potential', 'Easy To Groom',
       'General Health', 'Potential For Weight Gain', 'Size', 'Trainability',
       'Easy To Train', 'Intelligence', 'Potential For Mouthiness',
       'Prey Drive', 'Tendency To Bark Or Howl', 'Wanderlust Potential',
       'Physical Ne

In [223]:
dogs_pop_breed_der = dogs_pop_breed_der.rename(columns={
    "DISTRICT_CODE": "district_code",
    "English name": "dog_breed",
    "Dog Breed Group": "dog_breed_group",
    "Anzahl": "dog_count",
    "POP_VALUE": "population",
    "POP_DENSITY": "population_density",
    "AREA": "area_km2",
    "AVG_AGE": "avg_age",
    "Dog Size": "dog_size",
    "Adaptability": "adaptability",
    "All Around Friendliness": "friendliness",
    "Health And Grooming Needs": "health_needs",
    "Trainability": "trainability",
    "Exercise Needs": "exercise_needs",
    "DOG_DENSITY": "dog_density"
})

dogs_pop_breed_der = dogs_pop_breed_der.groupby(["district_code", "dog_breed"]).agg({
    "dog_breed_group": "first",
    "dog_count": "sum",
    "population": "first",
    "population_density": "first",
    "area_km2": "first",
    "avg_age": "first",
    "dog_size": "first",
    "adaptability": "first",
    "friendliness": "first",
    "health_needs": "first",
    "trainability": "first",
    "exercise_needs": "first",
    "dog_density": "sum"
}).reset_index()


In [224]:
dogs_clean = dogs_pop_breed_der[[
    "district_code",
    "dog_breed",
    "dog_breed_group",
    "dog_count",
    "population",
    "population_density",
    "area_km2",
    "avg_age",
    "dog_size",
    "adaptability",
    "friendliness",
    "health_needs",
    "trainability",
    "exercise_needs",
    "dog_density"
]]


In [225]:
dogs_clean

,district_code,dog_breed,dog_breed_group,dog_count,population,population_density,area_km2,avg_age,dog_size,adaptability,friendliness,health_needs,trainability,exercise_needs,dog_density
0,90100,Afghan Hound,Hound Dogs,6,16.290,5678.38,2.87,49.2,Very Large,4.0,4.67,2.0,3.0,4.0,0.37
1,90100,American Pit Bull Terrier,Terrier Dogs,2,16.290,5678.38,2.87,49.2,Very Large,2.4,3.67,3.4,3.8,4.0,0.12
2,90100,American Shepherd,None,1,16.290,5678.38,2.87,49.2,None,NaN,NaN,NaN,NaN,NaN,0.06
3,90100,American Staffordshire Terrier,Terrier Dogs,21,16.290,5678.38,2.87,49.2,Very Large,2.2,3.33,2.8,4.0,3.0,1.29
4,90100,Australian Shepherd,Herding Dogs,13,16.290,5678.38,2.87,49.2,Very Large,2.8,4.67,2.2,4.4,5.0,0.80
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3570,92300,Whippet,Hound Dogs,77,94.469,2946.47,32.06,45.1,Large,3.2,4.67,2.6,3.6,5.0,0.82
3571,92300,White Swiss Shepherd,Herding Dogs,45,94.469,2946.47,32.06,45.1,Very Large,3.2,4.00,3.4,4.6,5.0,0.48
3572,92300,Xoloitzcuintli,Companion Dogs,1,94.469,2946.47,32.06,45.1,Large,3.0,3.33,3.0,4.2,3.0,0.01
3573,92300,Yakutian Laika,Working Dogs,6,94.469,2946.47,32.06,45.1,Very Large,2.6,4.33,3.2,3.4,5.0,0.06


In [226]:
dogs_clean.to_csv('dogs_in_vienna.csv', index = False)

In [227]:
dogs_clean[dogs_clean["dog_breed_group"].isnull()].groupby("dog_breed").sum("dog_count").sort_values(by="dog_count", ascending=False)

,district_code,dog_count,population,population_density,area_km2,avg_age,adaptability,friendliness,health_needs,trainability,exercise_needs,dog_density
dog_breed,,,,,,,,,,,,
Unknown,2097600,98707,1717.084,230265.58,414.87,1007.7,0.0,0.0,0.0,0.0,0.0,1196.52
Podenco Canario,2006900,251,1686.983,211548.94,413.26,964.5,0.0,0.0,0.0,0.0,0.0,3.69
Elo,1461300,221,1202.033,147597.06,346.87,702.4,0.0,0.0,0.0,0.0,0.0,2.72
Bobtail,1369000,171,1262.653,117545.43,379.56,658.7,0.0,0.0,0.0,0.0,0.0,2.02
King Charles Spaniel,1276000,149,1177.037,125080.84,330.36,618.1,0.0,0.0,0.0,0.0,0.0,1.68
...,...,...,...,...,...,...,...,...,...,...,...,...
Petit Bleu de Gascogne,91200,4,88.515,10923.39,8.10,42.9,0.0,0.0,0.0,0.0,0.0,0.05
American Pocket Bully,184500,3,256.426,4529.64,134.36,87.5,0.0,0.0,0.0,0.0,0.0,0.02
Blue Griffon of Gascony,91800,2,47.632,7504.41,6.35,44.2,0.0,0.0,0.0,0.0,0.0,0.04
